In [267]:
from neo4j import GraphDatabase
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from flask import jsonify
from difflib import SequenceMatcher
import os
# from sklearn.metrics import pairwise_distances

Connect to Neo4j database


In [269]:
host = os.environ['HOST']
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]

conn = GraphDatabase.driver(host,auth=(username,password))


Input the title

In [467]:
inputText = "phantom"

*Processing......*

In [482]:
def run_recommendation(inputText):

    with conn.session() as session:
        res = "MATCH (a:Anime) RETURN a.title AS title, a.genre AS genre, a.rating as rating,a.studio AS studio, a.show_type AS show_type"
        # for i in res:
        #     print(i)
        query = session.run(res)

        data = [(row['title'],row['genre'],row['rating'],row['studio'],row['show_type'])for row in query]

        empList = list()
        for title,genre,rating,studio,show_type in data:
            genre_split = [i for i in genre]
            empList.append((title,", ".join(genre_split),rating,studio,show_type))

        genres = [genre for _, genre, _, _ ,_ in empList]

        count_vec = CountVectorizer(binary=True)
        matrix = count_vec.fit_transform(genres).todense()
        np_array = np.asarray(matrix)

        similarity_matrix = cosine_similarity(np_array)
        title_index = None
        similarity_scores = None

        for i in range(len(empList)):
            title = empList[i][0]
           
            sim = SequenceMatcher(None,inputText,title).ratio()

            if sim > 0.4:
                title_index = i
                break
     

        if title_index is not None:
            similarity_scores = similarity_matrix[title_index]
            top_indices = np.argsort(similarity_scores)[::-1]
            
            recommendation = [[list(empList[i]),similarity_scores[i]*100] for i in top_indices if similarity_scores[i]>0.7]
          
            
            res = {
                "status" : 200,
                "body" : recommendation
            }
        else:
            res = {
                "status" : 400,
                "body" : "Not Found"
            }
        return res

Results

In [483]:
run_recommendation(inputText)   

# sim = SequenceMatcher(None,"Call of the Night","Call Of") .ratio()


# print(sim)

{'status': 200,
 'body': [[['Phantom of the Idol',
    'Comedy, Supernatural',
    6.75,
    ' Studio Gokumi',
    'TV Show'],
   99.99999999999997],
  [['Osomatsu-san: Hipipo-Zoku to Kagayaku Kajitsu',
    'Comedy',
    6.42,
    'Pierrot',
    'Movie'],
   70.71067811865474],
  [['Chibiham, Juicy & Me', 'Comedy', 'N/A', 'N/A', 'Movie'],
   70.71067811865474],
  [['Musasi-no', 'Comedy', 4.69, ' A-Real', 'TV Show'], 70.71067811865474]]}

In [451]:
sim = SequenceMatcher(None,"drift","Drifting Home") .ratio()


print(sim)

0.4444444444444444
